In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import requests
import re
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
from geopy.geocoders import Nominatim
import time

In [5]:
def hotelazo(ciudad,check_in,check_out):
    opciones=Options()

    opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
    opciones.add_experimental_option('useAutomationExtension', False)

    opciones.headless=False    # si True, no aperece la ventana (headless=no visible)

    opciones.add_argument('--start-maximized')         # comienza maximizado

    #opciones.add_argument('--incognito')              # incognito

    url='https://www.atrapalo.com/'
    PATH= './chromedriver'
    driver = webdriver.Chrome(PATH, options=opciones)


    driver.get(url)
    time.sleep(3)

    cookies=driver.find_element_by_xpath('//*[@id="CybotCookiebotDialogBodyButtonAccept"]')
    cookies.click()

    hoteles=driver.find_element_by_xpath('//*[@id="content"]/section[1]/div/ul/li[5]/a/span')
    hoteles.click()

    time.sleep(3)

    buscar=driver.find_element_by_xpath('//*[@id="search_engine_HOT_destiny"]')
    buscar.send_keys(ciudad)
    buscar.submit()

    time.sleep(3)

    fecha=driver.find_element_by_xpath('//*[@id="searchDates"]')
    fecha.click()

    entrada=driver.find_element_by_xpath('//*[@id="finderContainer"]/div/form/div/div[1]/div[2]/div/div[1]/label')
    entrada.click()

    mes_1=driver.find_element_by_class_name('month1')

    for m in mes_1.find_elements_by_tag_name('td'):

        if m.text== check_in:
            m.click()

        elif m.text== check_out:
            m.click()


    aceptar=driver.find_element_by_xpath('//*[@id="submitFormButton"]')
    aceptar.click()

    texto_columnas = driver.find_element_by_class_name('hotels-list.box-resultados')

    #Sacamos nombre hotel
    nombre = []
    while range(2):
        time.sleep(5)
        texto_hoteles = driver.find_elements_by_class_name('openFontSemiBold')
        time.sleep(5)
        texto_hoteles = texto_hoteles[:4]
        for i in texto_hoteles:
                nombre.append(i.text)
        break

    ## Sacar Precio

    time.sleep(6)
    texto_precio = texto_columnas.find_elements_by_class_name('value')
    texto_precio = texto_precio[0:4]

    precio = []
    for i in texto_precio:
        precio.append(i.text)

    ## Sacar dirección

    hotel = driver.find_elements_by_class_name('product-name')

    direcc = []
    for i in hotel:
        primero = i.get_attribute("href")
        direcc.append(primero)

    direcc = direcc[0:4]

    driver.quit()

    time.sleep(5)
    address = []
    for link in direcc:
        driver = webdriver.Chrome("./chromedriver")
        driver.get(link)
        time.sleep(3)
        calle = driver.find_element_by_css_selector("span.app-address")
        calle = calle.text
        address.append(calle)
        driver.quit()
    '''
    latitud = []
    longitud = []
    for i in address:
        geolocator = Nominatim(user_agent="edu", timeout=5)
        location = geolocator.geocode(i)
        lat = location.latitude
        lon = location.longitude
        latitud.append(lat)
        longitud.append(lon)'''

    
    df = pd.DataFrame(list(zip(nombre,precio,address)), columns = ['Nombre','Precio','Direccion'])
    return df

In [7]:
hotelazo("Lanzarote","19","22")

,Nombre,Precio,Direccion
0,Hotel Los Zocos Club Resort,111€,"Av. Islas Canarias, 15, Costa Teguise 35508, L..."
1,Hotel Club Playa Blanca,95€,"C/ Faro Pechiguera, 2, Playa Blanca - Lanzarot..."
2,Hotel Meliá Salinas Inspirado Para Adultos,160€,"Av/ Islas Canarias, s/n, Costa Teguise 35509, ..."
3,Hotel Barceló Teguise Beach - Adults Only,139€,"Paseo Maritimo, s/n - 35508 , Costa Teguise 35..."


In [17]:
fecha = "2021-5-18"
fecha2 = fecha.split("-")
fecha2 = fecha2[2]
fecha2

'18'